# ***PROJETO FINAL***

## *INICIO DE TUDO, UPLOAD DF E LIMPEZA*

In [1]:
!pip install inflection

In [18]:
#FAZER UPLOAD DO ARQUIVO


from google import colab as cl


file_uploaded = cl.files.upload()

Saving zomato.csv to zomato (1).csv


In [ ]:
!pip install plotly

In [19]:
#imports para manipular o df
import pandas as pd
import inflection
import numpy as np

#import para criar gráficos
import plotly.express as px


#imports para criar mapas
import folium

In [20]:
#transformar CSV em DF

df = pd.read_csv('zomato (1).csv')

df = df.copy()

In [21]:
#CHECK NA
df.isna().sum()

#apagar os NA
df = df.dropna() #tira todos os espaços que não tem valor preenchido (no caso, os NaN)
df.isna().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                0
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

###Transformação do DataFrame

In [22]:

#RENOMEAÇÃO DE COLUNAS

def rename_columns(dataframe):
  df = dataframe.copy()
  title = lambda x: inflection.titleize(x)
  snakecase = lambda x: inflection.underscore(x)
  spaces = lambda x: x.replace(" ", "")
  cols_old = list(df.columns)
  cols_old = list(map(title, cols_old))
  cols_old = list(map(spaces, cols_old))
  cols_new = list(map(snakecase, cols_old))
  df.columns = cols_new
  return df

df = rename_columns(df)


#PREENCHIMENTO DO NOME DOS PAISES DE ACORDO COM SEUS RESPECTIVOS CÓDIGOS
def country_name(country_id):
  COUNTRIES = {
  1: "India",
  14: "Australia",
  30: "Brazil",
  37: "Canada",
  94: "Indonesia",
  148: "New Zeland",
  162: "Philippines",
  166: "Qatar",
  184: "Singapure",
  189: "South Africa",
  191: "Sri Lanka",
  208: "Turkey",
  214: "United Arab Emirates",
  215: "England",
  216: "United States of America",
  }

  return COUNTRIES[country_id]

#Criação do Tipo de Categoria de Comida FEITA PELO DS SENIOR

def create_price_tye(price_range):
  if price_range == 1:
    return "cheap"
  elif price_range == 2:
    return "normal"
  elif price_range == 3:
    return "expensive"
  else:
    return "gourmet"



#CRIAÇÃO DO NOME DAS CORES CRIADO PELO DS SENIOR
def color_name(color_code):
  COLORS = {
  "3F7E00": "darkgreen",
  "5BA829": "green",
  "9ACD32": "lightgreen",
  "CDD614": "orange",
  "FFBA00": "red",
  "CBCBC8": "darkred",
  "FF7800": "darkred",
  }

  return COLORS[color_code]

#CATEGORIZAR TODOS OS RESTAURANTES PO UM TIPO DE CULINARIA
df['cuisines'] = df.loc[:, 'cuisines'].apply(lambda x: x.split(',')[0])

#adicionar coluna 'country' Substituindo os códigos dos países pelos seus respectivos nomes
df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x))

#adicionar coluna color_name para gerar o mapa com as cores
df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x))

#foram excluidos os respectivos tipos de culinaria por não apresentarem informações completas na hora de gerar gráfico
df = df.drop(df[df["cuisines"] == "Drinks Only"].index)
df = df.drop(df[df["cuisines"] == "Mineira"].index)



## *Geral*

### 1- Quantos restaurantes únicos estão registrados?

In [ ]:
df['restaurant_id'].nunique()


6929

### 2- Quantos países únicos estão registrados?

In [ ]:
df['country'].nunique()


15

### 3- Quantas cidades únicas estão registradas?

In [ ]:
df['city'].nunique()



125

### 4- Qual o total de avaliações feitas?

In [ ]:
df['votes'].sum()



4638535

### 5- Qual o total de tipos de culinária registrados?

In [ ]:
df['cuisines'].nunique()


165

## *País*

In [ ]:
#função para graficos de barras com as métricas de paises
def graph_bar(metric):
  fig= px.bar(df_aux, x='country', y=metric)
  return fig

### 1. Qual o nome do país que possui mais cidades registradas? (p/ dashboard por qtd de cidades registradas por pais)


In [14]:
df.loc[:, ['country', 'city']].groupby('country').nunique('city').sort_values('city', ascending = False).reset_index().iloc[0,0]


'India'

### 2. Qual o nome do país que possui mais restaurantes registrados? (p/ dashboard por qtd de  restaurantes registradas por pais)

In [25]:
df.loc[: , ['country', 'restaurant_id']].groupby('country').nunique('restaurant_id').sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]




'India'

### 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [29]:
linhas = df['price_range'] == 4
cols = ['country', 'restaurant_id']
df.loc[linhas, cols].groupby('country').nunique('restaurant_id').reset_index().sort_values('restaurant_id', ascending=False).iloc[0,0]


'United States of America'

### 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [31]:
df.loc[:, ['country', 'cuisines']].groupby('country').nunique('cuisines').reset_index().sort_values('cuisines', ascending=False).iloc[0,0]



'India'

### 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?


In [32]:
df.loc[:, ['country', 'aggregate_rating']].groupby('country').sum('aggregate_rating').reset_index().sort_values('aggregate_rating', ascending=False).iloc[0,0]


'India'

### 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [33]:
linhas = df['has_online_delivery'] == 1
cols = ['country', 'restaurant_id']

df.loc[linhas, cols].groupby('country').nunique('restaurant_id').reset_index().sort_values('restaurant_id', ascending=False).iloc[0,0]


'India'

### 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [34]:
linhas = df['has_table_booking'] == 1
cols = ['country', 'restaurant_id']

df.loc[linhas, cols].groupby('country').count().reset_index().sort_values('restaurant_id', ascending=False).iloc[0,0]


'India'

### 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [38]:
df.loc[:, ['country', 'votes']].groupby('country').mean('votes').reset_index().sort_values('votes', ascending=False).iloc[0,0]


'Indonesia'

### 9. Qual o nome do país que possui, na média, a maior nota média registrada?


In [37]:
df.loc[:, ['country', 'aggregate_rating']].groupby('country').mean('aggregate_rating').reset_index().sort_values('aggregate_rating', ascending=False).iloc[0,0]


'Indonesia'


### 10. Qual o nome do país que possui, na média, a menor nota média registrada?

In [36]:
df.loc[:, ['country', 'aggregate_rating']].groupby('country').mean('aggregate_rating').reset_index().sort_values('aggregate_rating', ascending=True).iloc[0,0]


'Brazil'

### 11. Qual a média de preço de um prato para dois por país?

In [35]:
df.loc[:, ['country', 'average_cost_for_two']].groupby('country').mean().reset_index().sort_values('average_cost_for_two', ascending=False).iloc[0,0]


'Indonesia'

## *Cidade*

In [ ]:
#função para graficos de barras das cidades

def graph_bar(metric):
  fig = px.bar(df_aux, x='city', y=metric)
  return fig

###  1. Qual o nome da cidade que possui mais restaurantes registrados?

In [40]:
df.loc[:, ['city', 'restaurant_id']].groupby('city').nunique('restaurant_id').sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]


'Abu Dhabi'

### 2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4?

In [39]:
linhas = df['aggregate_rating'] >= 4
cols = ['city', 'restaurant_id']

df.loc[linhas, cols].groupby('city').nunique('restaurant_id').sort_values('restaurant_id', ascending=False).reset_index().iloc[0,0]

'London'

### 3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5?

In [41]:
linhas = df['aggregate_rating'] <= 2.5
cols = ['city', 'restaurant_id']

df.loc[linhas, cols].groupby('city').nunique('restaurant_id').reset_index().sort_values('restaurant_id', ascending=False).iloc[0,0]

'Gangtok'

### 4. Qual o nome da cidade que possui o maior valor médio de um prato para dois?


In [42]:
df.loc[:, ['city', 'average_cost_for_two']].groupby('city').mean('average_cost_for_two').reset_index().sort_values('average_cost_for_two', ascending=False).iloc[0,0]


'Adelaide'

### 5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas?

In [43]:
df.loc[:, ['city', 'cuisines']].groupby('city').nunique().sort_values('cuisines', ascending = False).reset_index().iloc[0,0]

'Birmingham'

### 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas?

In [44]:
linhas = df['has_table_booking'] == 1
df.loc[linhas, ['city', 'restaurant_id']].groupby('city').count().sort_values('restaurant_id', ascending = False).reset_index().iloc[0,0]

'Bangalore'

### 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que estão fazendo entregas?

In [46]:
linhas = df['is_delivering_now'] == 1
df.loc[linhas, ['city', 'restaurant_id']].groupby('city').count().sort_values('restaurant_id', ascending = False).reset_index().iloc[0,0]


'Aurangabad'

### 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?

In [48]:
df.loc[df['has_online_delivery'] == 1 , ['restaurant_id', 'city']].groupby('city').count().sort_values('restaurant_id', ascending = False).reset_index().iloc[0,0]


'Abu Dhabi'

## *Restaurantes*

### 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?

In [49]:
cols = ['restaurant_name', 'restaurant_id', 'votes']

df.loc[:, cols].reset_index().sort_values(['votes', 'restaurant_id'], ascending= [False, True]).iloc[0,1]

'Bawarchi'


### 2. Qual o nome do restaurante com a maior nota média?

In [50]:
cols= ['restaurant_name', 'restaurant_id', 'aggregate_rating']

df.loc[:, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True]).reset_index().iloc[0,1]


'Indian Grill Room'

### 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?

In [51]:
cols = ['restaurant_name', 'restaurant_id', 'average_cost_for_two']
df.loc[:, cols].sort_values(['average_cost_for_two', 'restaurant_id'], ascending = [False, True]).reset_index().iloc[0,1]


"d'Arry's Verandah Restaurant"

### 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?

In [52]:

linhas = df['cuisines'] == 'Brazilian'
cols = ['restaurant_name', 'restaurant_id', 'aggregate_rating']
df.loc[linhas, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending = [True, True]).reset_index().iloc[0,1]


'Loca Como tu Madre'

### 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?

In [53]:

linhas = (df['cuisines'] == 'Brazilian') & (df['country'] == 'Brazil')
cols = ['restaurant_name', 'restaurant_id', 'aggregate_rating']
df.loc[linhas, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False,True]).reset_index().iloc[0,1]



'Braseiro da Gávea'

### 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?

In [56]:
df.loc[: , ['has_online_delivery', 'votes']].groupby('has_online_delivery').mean().sort_values('votes', ascending=False).reset_index()


,has_online_delivery,votes
0,1,843.027516
1,0,494.539839


### 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas?

In [57]:
df.loc[:, ['has_table_booking', 'average_cost_for_two']].groupby('has_table_booking').mean().sort_values('average_cost_for_two', ascending = False).reset_index()


,has_table_booking,average_cost_for_two
0,1,62931.512821
1,0,3461.561204


### 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?


In [60]:
linhas = (df['cuisines'] == 'Japanese') & (df['country'] == 'United States of America') | (df['cuisines'] == 'BBQ') & (df['country'] == 'United States of America')
df.loc[linhas, ['cuisines', 'average_cost_for_two']].groupby('cuisines').mean().sort_values('average_cost_for_two', ascending=False).reset_index()


,cuisines,average_cost_for_two
0,Japanese,56.406250
1,BBQ,39.642857


## *Tipos de Culinária*

In [65]:
#função que retorna valor de acordo com a culinaria desejada e media de avaliação desejada(maior ou menos)

def metric(cuisines, top):
  #top == False(maior média) / True(menor média)
  cols = [
        "restaurant_id",
        "restaurant_name",
        "country",
        "city",
        "cuisines",
        "average_cost_for_two",
        "currency",
        "aggregate_rating",
        "votes",
    ]

  linhas = df['cuisines'] == cuisines
  resultado = df.loc[linhas, cols].sort_values(['aggregate_rating', 'restaurant_id'], ascending=[top,True])


  return resultado




### 1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?

In [72]:
metric('Italian', False).iloc[0,1]



'Darshan'

### 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?

In [71]:
metric('Italian', True).iloc[0,1]

'Avenida Paulista'

### 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?


In [69]:
metric('American', False).iloc[0,1]

'Burger & Lobster'

### 4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?

In [80]:
metric('American', True).iloc[0,1]

'Alston Bar & Beef'

### 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?

In [79]:
metric('Arabian', False).iloc[0,1]

'Mandi@36'

### 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?

In [78]:
metric('Arabian', True).iloc[0,1]

'Raful'

### 7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a maior média de avaliação?

In [77]:
metric('Japanese', False).iloc[0,1]

'Sushi Samba'

### 8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome do restaurante com a menor média de avaliação?

In [76]:
metric('Japanese', True).iloc[0,1]

'Banzai Sushi'


### 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?

In [75]:
metric('Home-made', False).iloc[0,1]

'Kanaat Lokantası'

### 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?

In [73]:
metric('Home-made', True).iloc[0,1]

'GurMekan Restaurant'

### 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?

In [81]:
df.loc[:, ['cuisines', 'average_cost_for_two']].sort_values('average_cost_for_two', ascending=False).reset_index().head(1)


,index,cuisines,average_cost_for_two
0,385,Modern Australian,25000017


### 12. Qual o tipo de culinária que possui a maior nota média?

In [86]:
df.loc[:, ['cuisines', 'aggregate_rating']].sort_values('aggregate_rating', ascending=False).reset_index().head(1)

,index,cuisines,aggregate_rating
0,1135,American,4.9


### 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?

In [89]:
linhas= (df['has_online_delivery'] == 1) & (df['is_delivering_now'] == 1)
df.loc[linhas, ['cuisines', 'restaurant_id']].groupby('cuisines').count().sort_values('restaurant_id', ascending=False).reset_index().head(1)

,cuisines,restaurant_id
0,North Indian,343
